# In this practice session, we will learn to code Decision Tree Regression. 
# We will perform the following steps to build a simple classifier using the popular Beer dataset.

 
 
  - **Data Preprocessing**

    - Importing the libraries.
    - Importing dataset.
    - Dealing with the categorical variable.
    - Classifying dependent and independent variables.
    - Splitting the data into a training set and test set.
    - Feature scaling.
 

  -  **Decision Tree Regression**

    - Create a Decision Tree Regression.
    - Feed the training data to the regression.
    - Predicting the score for the test set.
    - Using the rmse to calculate the performance.

Decision Tree Regression is a machine learning model that uses a decision tree for predictive analysis. Decision Tree builds regression models in the form of a tree structure. Decision trees are one of the simplest and most common ways of modeling outcomes based on decisions.

In machine learning, Decision Trees start by identifying the best feature for splitting the dataset which then finds the subsets that has the best possible value for the best feature. The dataset is split based on decisions until a level of optimization is reached. 

# Load the Dependencies

In [1]:
import ipywidgets as widgets
from IPython.display import display

style = {'description_width': 'initial'}

In [2]:
#1 Importing essential libraries
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Load the Dataset

In [3]:
#2 Importing the dataset

file_name = 'dataset/beer_data.csv'
dataset = pd.read_csv(file_name)

In [4]:
#Displaying the dataset
dataset.head(8)

,beer_beerid,review_profilename,review_overall
0,48215,stcules,3.0
1,52159,oline73,3.0
2,52159,alpinebryant,3.0
3,52159,rawthar,4.0
4,52159,RangerClegg,3.5
5,58046,mikedrinksbeer2,4.5
6,58046,dbmernin83,4.0
7,58046,Blakaeris,4.5


In [5]:
print(f"Dataset has {dataset.shape[0]} rows and {dataset.shape[1]} columns.")

Dataset has 475984 rows and 3 columns.


## Feature Engineering

#### Drop Nulls and Fill Nulls Based on Mean

In [6]:
#check nulls..

dataset.isnull().sum()

beer_beerid             0
review_profilename    100
review_overall          0
dtype: int64

In [7]:
dataset = dataset[~dataset['Cellar Temperature'].isna()]
dataset.reset_index(inplace=True, drop=True)

KeyError: 'Cellar Temperature'

In [ ]:
dataset['ABV'].fillna(dataset['ABV'].mean(), inplace=True)

In [ ]:
dataset['Ratings'] = dataset['Ratings'].apply(lambda x : np.float32(x.replace(",", "")))

In [ ]:
# Dealing with the categorical data

# Spliting Cellar Temperature into Maximum and Minimum based on the given data and converting the type from str to int

dataset.loc[:, 'Minimum_Cellar_Temp'] = dataset['Cellar Temperature'].apply(lambda x : int(str(x).split('-')[0].strip()))
dataset.loc[:, 'Maximum_Cellar_Temp'] = dataset['Cellar Temperature'].apply(lambda x : int(str(x).split('-')[1].strip()))

In [ ]:
dataset.drop('Cellar Temperature', inplace=True, axis=1)
dataset.columns.tolist()

In [ ]:
#3 classify dependent and independent variables
X = dataset[[col for col in dataset.columns if col not in ('Score')]].values  #independent variables 
y = dataset['Score'].values  #dependent variable 

In [ ]:
print("\nIdependent Variables :\n\n", X[:5])
print("\nDependent Variable (Score):\n\n", y[:5])

# Create Train and Test Sets

In [ ]:
#4 Creating training set and testing set
from sklearn.model_selection import train_test_split
test_size = widgets.FloatSlider(min=0.01, max=0.6, value=0.2, description="Test Size :", tooltips=['Usually 20-30%'])
display(test_size)

In [ ]:
#Divide the dataset into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=test_size.value, random_state = 0) 

In [ ]:
print("Training Set :\n----------------\n")
print("X = \n", X_train[:5])
print("y = \n", y_train[:5])

print("\n\nTest Set :\n----------------\n")
print("X = \n",X_test[:5])
print("y = \n", y_test[:5])

In [ ]:
print(f"Shape of Training set is {X_train.shape}")
print(f"Shape of Testing set is {X_test.shape}")

# Apply Decision Tree Regression 

In [ ]:
# import decision tree library
from sklearn.tree import DecisionTreeRegressor

# configure params for the model.
max_feat_wig = widgets.ToggleButtons(options=['log2', 'sqrt', 'auto'],
                                    description='Number of features for the best split :',
                                    disabled=False,
                                    style=style)

display(max_feat_wig)

max_depth_wig = widgets.Dropdown(options=[10, 20, 30, 50],
                            description='The maximum depth of the Tree. :',
                            style=style)

display(max_depth_wig)

min_split_wig = widgets.Dropdown(options=[100, 200, 300, 500],
                            description='The maximum depth of the Tree. :',
                            style=style)

display(min_split_wig)

# Predict and Evaluate the Model 

In [ ]:
# Train the Regressor with training set
regressor = DecisionTreeRegressor(max_features=max_feat_wig.value,
                                  max_depth=max_depth_wig.value,
                                  min_samples_split=min_split_wig.value)

#fit the linear model
regressor.fit(X_train, y_train)

#7 predict the outcome of test sets
y_Pred = regressor.predict(X_test)
print("\nPredictions = ", y_Pred)

In [ ]:
# Calculating score from Root Mean Log Squared Error
def rmlse(y_test, y_pred):
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    score = 1 - error
    return score

# Printing the score
print("\n----------------------------\nRMLSE Score = ", rmlse(y_test, y_Pred))

#9 Comparing Actual and Predicted Salaries for he test set
print("\nActual vs Predicted Scores \n------------------------------\n")
error_df = pd.DataFrame({"Actual" : y_test,
                         "Predicted" : y_Pred,
                         "Abs. Error" : np.abs(y_test - y_Pred)})

error_df

# Actual vs. Predicted 

In [ ]:
#Plotting Actual observation vs Predictions
plt.scatter(y_test, y_Pred, s = 70)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.legend();
plt.grid();
plt.show();